In [1]:
import numpy as np
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random

# Configuration des seeds pour la reproductibilité
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

config = {
    "num_episodes_deep": 700,
    "epsilon": 1.0,
    "epsilon_decay": 0.999,
    "learning_rate": 0.1,
    "discount_factor": 0.99
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de : {device}")

Utilisation de : cpu


In [2]:
class DQN(nn.Module):
    def __init__(self, input_size, output_size):
        super(DQN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        return self.network(x)

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)

In [3]:
def train_deep(env, config):
    # Initialisation des réseaux de neurones
    policy_net = DQN(env.observation_space.n, env.action_space.n).to(device)  # Réseau principal
    target_net = DQN(env.observation_space.n, env.action_space.n).to(device)  # Réseau cible
    target_net.load_state_dict(policy_net.state_dict())  # Copie des poids initiaux

    optimizer = optim.Adam(policy_net.parameters(), lr=0.001)  # Optimiseur
    memory = ReplayBuffer(10000)  # Mémoire pour stocker les expériences
    batch_size = 64  # Taille des batchs d'apprentissage

    rewards_per_episode = np.zeros(config["num_episodes_deep"])
    success_rate = []
    success_window = []
    epsilon = config["epsilon"]  # Probabilité d'exploration

    for episode in range(config["num_episodes_deep"]):
        state = env.reset()[0]  # Reset de l'environnement
        terminated = False
        truncated = False
        episode_reward = 0
        episode_success = False

        # Boucle d'un épisode
        while not terminated and not truncated:
            # Conversion de l'état en tensor one-hot
            state_tensor = torch.zeros(env.observation_space.n)
            state_tensor[state] = 1
            state_tensor = state_tensor.to(device)

            # Sélection de l'action (exploration ou exploitation)
            if random.random() < epsilon:  # Exploration
                action = env.action_space.sample()
            else:  # Exploitation
                with torch.no_grad():
                    action = policy_net(state_tensor).argmax().item()

            # Exécution de l'action
            next_state, reward, terminated, truncated, _ = env.step(action)
            episode_reward += reward

            if reward > 0:
                episode_success = True

            # Préparation du prochain état
            next_state_tensor = torch.zeros(env.observation_space.n)
            next_state_tensor[next_state] = 1

            # Stockage de l'expérience dans la mémoire
            memory.push(state_tensor, action, reward, next_state_tensor, terminated)

            # Apprentissage si assez d'expériences stockées
            if len(memory) >= batch_size:
                batch = memory.sample(batch_size)
                states, actions, rewards, next_states, dones = zip(*batch)

                # Préparation des tensors
                states = torch.stack(states).to(device)
                actions = torch.tensor(actions).to(device)
                rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
                next_states = torch.stack(next_states).to(device)
                dones = torch.tensor(dones, dtype=torch.float32).to(device)

                # Calcul des Q-values actuelles et cibles
                current_q_values = policy_net(states).gather(1, actions.unsqueeze(1))
                next_q_values = target_net(next_states).max(1)[0].detach()
                target_q_values = rewards + (1 - dones) * config["discount_factor"] * next_q_values

                # Optimisation
                loss = nn.MSELoss()(current_q_values.squeeze(), target_q_values)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            state = next_state

        # Mise à jour du réseau cible périodiquement
        if episode % 10 == 0:
            target_net.load_state_dict(policy_net.state_dict())

        # Enregistrement des statistiques
        rewards_per_episode[episode] = episode_reward
        success_window.append(1 if episode_success else 0)

        if success_window:
            if len(success_window) > 100:
                success_window.pop(0)
            current_success_rate = sum(success_window) / len(success_window) * 100
            success_rate.append(current_success_rate)
        else:
            success_rate.append(0)

        # Décroissance du epsilon
        epsilon = max(epsilon * config["epsilon_decay"], 0.01)

        # Affichage périodique des performances
        if (episode + 1) % 100 == 0:
            avg_success = sum(success_window) / len(success_window) * 100
            print(f"Deep Q-Learning - Épisode {episode + 1}, Taux de réussite: {avg_success:.2f}%")

    return policy_net, rewards_per_episode, success_rate


In [4]:
def evaluate_deep(env, policy_net, num_episodes=100):
    """
    Évalue les performances du Deep Q-learning
    """
    total_rewards = []

    for _ in range(num_episodes):
        state = env.reset()[0]
        terminated = False
        truncated = False
        episode_reward = 0

        while not terminated and not truncated:
            state_tensor = torch.zeros(env.observation_space.n)
            state_tensor[state] = 1
            state_tensor = state_tensor.to(device)

            with torch.no_grad():
                action = policy_net(state_tensor).argmax().item()

            state, reward, terminated, truncated, _ = env.step(action)
            episode_reward += reward

        total_rewards.append(episode_reward)

    mean_reward = np.mean(total_rewards)
    std_reward = np.std(total_rewards)
    print(f"Évaluation sur {num_episodes} épisodes:")
    print(f"Récompense moyenne: {mean_reward:.2f} ± {std_reward:.2f}")

    return mean_reward

In [5]:
print("\nEntraînement du Deep Q-learning...")
env = gym.make('Taxi-v3')
deep_q_net, deep_rewards, deep_success_rate = train_deep(env, config)


Entraînement du Deep Q-learning...
Deep Q-Learning - Épisode 100, Taux de réussite: 9.00%
Deep Q-Learning - Épisode 200, Taux de réussite: 29.00%
Deep Q-Learning - Épisode 300, Taux de réussite: 79.00%
Deep Q-Learning - Épisode 400, Taux de réussite: 97.00%
Deep Q-Learning - Épisode 500, Taux de réussite: 100.00%
Deep Q-Learning - Épisode 600, Taux de réussite: 100.00%
Deep Q-Learning - Épisode 700, Taux de réussite: 100.00%
Deep Q-Learning - Épisode 800, Taux de réussite: 100.00%
Deep Q-Learning - Épisode 900, Taux de réussite: 100.00%
Deep Q-Learning - Épisode 1000, Taux de réussite: 100.00%
Deep Q-Learning - Épisode 1100, Taux de réussite: 100.00%
Deep Q-Learning - Épisode 1200, Taux de réussite: 100.00%
Deep Q-Learning - Épisode 1300, Taux de réussite: 100.00%
Deep Q-Learning - Épisode 1400, Taux de réussite: 100.00%
Deep Q-Learning - Épisode 1500, Taux de réussite: 100.00%
Deep Q-Learning - Épisode 1600, Taux de réussite: 100.00%
Deep Q-Learning - Épisode 1700, Taux de réussite: 